 # Teste Melhor Envio
 ### Requisitos
###### Processar o arquivo de log, extrair informações e salvá-las no banco de dados.
###### Gerar um relatório para cada descrição abaixo, em formato csv:
###### Requisições por consumidor;
###### Requisições por serviço;
###### Tempo médio de request , proxy e gateway por serviço.
###### Documentar passo a passo de como executar o teste através de um arquivo
###### README.md .
###### Efetue o commit de todos os passos do desenvolvimento em um git público de sua
###### preferência e disponibilize apenas o link para o repositório.

In [226]:
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv


def ImportLog():
    return pd.read_json('logs.txt', lines=True)


def OrganizaDataFrame(dados):
    # organizando o lantencia pois ele que usaremos para os relatorios
    dados['latencies']['service_id'] = dados['services'].id
    dados['latencies']['consumer_id'] = dados['authenticated_entity']
    dados['latencies']['client_ip'] = dados['client_ip']
    dados['latencies']['route'] = dados['routes'].id

    #dizendo par ao DataFrame que o id é o index de routes
    dados['routes'].set_index('id', inplace=True)

    # renomeando a coluna para que não tenha .
    dados['routes'].rename(columns={'service.id': 'service_id'}, inplace=True)

    # os pop's a seguir são para limpar variaveis que os dados se repetem
    # para descobrir quais são fiz um:
    # new_data_frame[nomeDoDataFrame][headerQueValidei].value_counts()
    dados['requests'].drop(['headers.user-agent', 'uri', 'querystring', 'headers.accept'], axis=1, inplace=True)
    dados['routes'].drop(['regex_priority', 'preserve_host', 'strip_path', 'protocols', 'paths', 'methods'],
                         axis=1, inplace=True)
    dados['services'].drop(['path', 'port', 'protocol', 'read_timeout', 'retries', 'write_timeout', 'connect_timeout'],
                           axis=1, inplace=True)
    return dados


def ArrumaJson(data_frame):
    # criação do novo dataframe com os dados convertidos de json para dataframe
    # iniciamente pensei em gravar todos esses dados no banco
    # porem fui informado que não era necessario
    new_data_frame = {
        'latencies': pd.json_normalize(data_frame.latencies),
        'requests': pd.json_normalize(data_frame.request),
        'responses': pd.json_normalize(data_frame.response),
        'authenticated_entity': pd.json_normalize(data_frame.authenticated_entity),
        'routes': pd.json_normalize(data_frame.route),
        'services': pd.json_normalize(data_frame.service),
        'client_ip': data_frame.client_ip,
        'upstream_uri': data_frame.upstream_uri,
        'started_at': data_frame.started_at
    }

    return OrganizaDataFrame(new_data_frame)


def RelatorioConsumidor(consumers):
    # organizar por consumidor
    relatorio_por_consumidor = consumers.groupby(by='consumer_id')
    csv = {}
    os.makedirs('customers', exist_ok=True)
    for customer, dados in relatorio_por_consumidor:
        csv[customer] = dados
        csv[customer].to_csv(f"./customers/{customer}.csv", sep=";")
    return relatorio_por_consumidor


def RelatorioServico(servico):
    # organizar por serviço
    relatorio_por_servico = new_data_frame['services'].groupby(by='id')
    csv = {}
    os.makedirs('services', exist_ok=True)
    for services, dados in relatorio_por_servico:
        csv[services] = dados
        csv[services].to_csv(f"./services/{dados.iloc[0]['name']}.csv", sep=";")
    return relatorio_por_servico


def RelatorioMediaLatencia(latencies):
    # organizar pelo consumidor com as médias
    relatorio_medias_consumidor = latencies.groupby('consumer_id').mean()
    relatorio_medias_consumidor.to_csv('relatorio_medias_consumidor.csv', sep=';')
    return relatorio_medias_consumidor


def CriarBase(base):
    engine = create_engine(f"mysql+pymysql://{os.getenv('USER_NAME')}:{os.getenv('USER_PASSWORD')}@"
                           f"{os.getenv('HOST_NAME')}/{os.getenv('DATABASE')}", echo=True)
    # para o proposito dessa atividade deixei para que ele recrie a base se ela já existir
    # acredito que esse não seja o recomendado, porem para o objetivo da tarefa é o mais indicado
    new_data_frame['routes'].to_sql('routes',engine, if_exists='replace', index=False, index_label='id')
    new_data_frame['requests'].to_sql('requests',engine, if_exists='replace', index=True, index_label='request')
    new_data_frame['latencies'].to_sql('latencies',engine, if_exists='replace', index=False, index_label='id')
    new_data_frame['services'].to_sql('services',engine, if_exists='replace', index=False, index_label='id')

if __name__ == '__main__':
    load_dotenv()
    data_frame = ImportLog()
    new_data_frame = ArrumaJson(data_frame)
    RelatorioConsumidor(new_data_frame['latencies'])
    RelatorioServico(new_data_frame['latencies'])
    RelatorioMediaLatencia(new_data_frame['latencies'])
    CriarBase(new_data_frame)


2022-06-16 06:27:30,895 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-06-16 06:27:30,896 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-06-16 06:27:30,897 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-06-16 06:27:30,898 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-06-16 06:27:30,899 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-06-16 06:27:30,900 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-06-16 06:27:30,902 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-06-16 06:27:30,903 INFO sqlalchemy.engine.Engine [generated in 0.00087s] {'table_schema': 'melhorenvio', 'table_name': 'routes'}
2022-06-16 06:27:30,906 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-06-16 06:27:30,906 INFO sqlalchemy.engine.Engine [cached since 0.003974s ago] {'table_sc

2022-06-16 06:28:04,875 INFO sqlalchemy.engine.Engine COMMIT
2022-06-16 06:28:05,170 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-06-16 06:28:05,171 INFO sqlalchemy.engine.Engine [cached since 34.27s ago] {'table_schema': 'melhorenvio', 'table_name': 'services'}
2022-06-16 06:28:05,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-16 06:28:05,173 INFO sqlalchemy.engine.Engine 
CREATE TABLE services (
	created_at BIGINT, 
	host TEXT, 
	id TEXT, 
	name TEXT, 
	updated_at BIGINT
)


2022-06-16 06:28:05,174 INFO sqlalchemy.engine.Engine [no key 0.00060s] {}
2022-06-16 06:28:05,627 INFO sqlalchemy.engine.Engine COMMIT
2022-06-16 06:28:05,687 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-16 06:28:06,228 INFO sqlalchemy.engine.Engine INSERT INTO services (created_at, host, id, name, updated_at) VALUES (%(created_at)s, %(host)s, %(id)s, %(name)s, %(updated_at)s)
2022-06-

In [235]:
new_data_frame['latencies']

,proxy,kong,request,service_id,consumer_id,client_ip,route
0,1836,8,1058,c3e86413-648a-3552-90c3-b13491ee07d6,72b34d31-4c14-3bae-9cc6-516a0939c9d6,75.241.168.121,0636a119-b7ee-3828-ae83-5f7ebbb99831
1,1727,10,2185,d035ffcf-914a-3007-b028-ae18f04d75b4,f643db14-a82d-30c2-8c13-889db1d0fcc2,88.248.178.118,7bedb816-c359-3b8c-a0f9-b03b46c63402
2,896,9,2129,a5bf08bd-c030-30d5-8009-83a8c30103bf,beceaa24-823b-3bf9-9ae6-c8dada26b264,56.54.196.221,9bb5b399-23b4-3c5a-b92b-2febdbcfa4e4
3,1209,12,1274,22f8e3a6-01f7-3264-b4b5-9d178df11d06,7ba24e1f-ed19-31b2-a4be-4114721d63af,164.83.216.199,2c31902e-00af-30aa-a04f-08c50ed55785
4,1708,20,2244,22f8e3a6-01f7-3264-b4b5-9d178df11d06,1696f3c4-7732-38f2-9ba8-0eeca8df05d7,154.78.181.43,2c31902e-00af-30aa-a04f-08c50ed55785
...,...,...,...,...,...,...,...
99995,1749,6,2377,eb1ce287-5797-3e45-b9a6-e6d51691257e,1b848a2b-8abf-352e-a08b-26e1af91fb7d,98.90.136.186,b5e18fb7-f7e6-38a6-b614-68acd2508ab2
99996,1915,10,1481,d035ffcf-914a-3007-b028-ae18f04d75b4,5f1ced25-0e7e-3939-8bd8-bf739b47ff47,43.14.93.160,7bedb816-c359-3b8c-a0f9-b03b46c63402
99997,1677,8,1418,d035ffcf-914a-3007-b028-ae18f04d75b4,98fff68a-e94f-3b46-8e22-c88c3739ef27,210.61.28.63,7bedb816-c359-3b8c-a0f9-b03b46c63402
99998,1264,18,2458,22f8e3a6-01f7-3264-b4b5-9d178df11d06,aac8d101-c4b5-3fd6-8833-b0fba8f36587,54.231.110.76,2c31902e-00af-30aa-a04f-08c50ed55785
